In [15]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
import xgboost as xgb

import pickle

In [2]:
urldata = pd.read_csv('urldata_nlp.csv')

In [3]:
def type_encoder(x):
    if x == 'malicious':
        return 1
    elif x == 'benign':
        return 0
    else:
        return x

urldata['target'] = urldata['type'].apply(lambda x: type_encoder(x))

In [4]:
urldata["clean_text"].dropna(inplace=True)
# convert the column to string
urldata['clean_text'] = urldata['clean_text'].astype(str)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(urldata["clean_text"], urldata["target"],test_size=0.2,shuffle=True)

In [6]:
#Tf-Idf
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train) 
X_test_vectors_tfidf = tfidf_vectorizer.transform(X_test)

In [8]:
adaB_tfidf = AdaBoostClassifier()
adaB_tfidf.fit(X_train_vectors_tfidf,y_train)
y_pred = adaB_tfidf.predict(X_test_vectors_tfidf)
y_prob = adaB_tfidf.predict_proba(X_test_vectors_tfidf)[:,1]
print(classification_report(y_test,y_pred))
print('Confusion Matrix:',confusion_matrix(y_test, y_pred))
 
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

              precision    recall  f1-score   support

           0       0.85      0.94      0.89    154905
           1       0.79      0.58      0.67     63141

    accuracy                           0.83    218046
   macro avg       0.82      0.76      0.78    218046
weighted avg       0.83      0.83      0.83    218046

Confusion Matrix: [[144846  10059]
 [ 26268  36873]]
AUC: 0.8545273780751845


In [9]:
filename = 'adaB_tfidf_model.sav'
pickle.dump(adaB_tfidf, open(filename, 'wb'))

In [10]:
xgb_tfidf= xgb.XGBClassifier(n_estimators= 1000)
xgb_tfidf.fit(X_train_vectors_tfidf,y_train)
y_pred = xgb_tfidf.predict(X_test_vectors_tfidf)
y_prob = xgb_tfidf.predict_proba(X_test_vectors_tfidf)[:,1]
print(classification_report(y_test,y_pred))
print('Confusion Matrix:',confusion_matrix(y_test, y_pred))
 
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

              precision    recall  f1-score   support

           0       0.92      0.96      0.94    154905
           1       0.88      0.80      0.84     63141

    accuracy                           0.91    218046
   macro avg       0.90      0.88      0.89    218046
weighted avg       0.91      0.91      0.91    218046

Confusion Matrix: [[148183   6722]
 [ 12734  50407]]
AUC: 0.9654118107787145


In [11]:
filename = 'xgB_tfidf_model.sav'
pickle.dump(xgb_tfidf, open(filename, 'wb'))

In [12]:
#FITTING THE CLASSIFICATION MODEL using random forest(tf-idf)
rf_tfidf=RandomForestClassifier(n_estimators=100)
rf_tfidf.fit(X_train_vectors_tfidf, y_train)  
#Predict y value for test dataset
y_predict = rf_tfidf.predict(X_test_vectors_tfidf)
y_prob = rf_tfidf.predict_proba(X_test_vectors_tfidf)[:,1]
print(classification_report(y_test,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test, y_predict))
 
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

              precision    recall  f1-score   support

           0       0.94      0.97      0.95    154905
           1       0.91      0.85      0.88     63141

    accuracy                           0.93    218046
   macro avg       0.92      0.91      0.92    218046
weighted avg       0.93      0.93      0.93    218046

Confusion Matrix: [[149537   5368]
 [  9567  53574]]
AUC: 0.9713607765339486


In [13]:
filename = 'rf_tfidf_model.sav'
pickle.dump(rf_tfidf, open(filename, 'wb'))

In [ ]:
svm_tfidf = svm.SVC(probability=True)
svm_tfidf.fit(X_train_vectors_tfidf, y_train)

#Predict y value for test dataset
y_predict = svm_tfidf.predict(X_test_vectors_tfidf)
y_prob = svm_tfidf.predict_proba(X_test_vectors_tfidf)[:,1]
print(classification_report(y_test,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test, y_predict))
 
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

In [ ]:
filename = 'svm_tfidf_model.sav'
pickle.dump(svm_tfidf, open(filename, 'wb'))

In [ ]:
filename = 'tfidf_vectorizer_model.sav'
pickle.dump(tfidf_vectorizer, open(filename, 'wb'))